In [1]:

!pip3 install pyspark==3.0.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 204.7/204.7 MB 2.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 198.6/198.6 kB 12.9 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.0.0-py2.py3-none-any.whl size=205044159 sha256=2aced1cef22e037bfe65e871fe9c55df74bbfa9c4bff8e11905a373c78ad0319
  Stored in directory: /root/.cache/pip/wheels/b1/bb/8b/ca24d3f756f2ed967225b0871898869db676eb5846df5adc56
Successfully built pyspark
  Attempting uninstall: py4j
    Found existing installation: py4j 0.10.9.7
    Uninstalling py4j-0.10.9.7:
      Successfully uninstalled py4j-0.10.9.7


In [2]:

from pyspark.sql import SparkSession
import pyspark.sql.functions as F
import pyspark.sql.types as t
from geopy.distance import geodesic
from math import sqrt

In [3]:

spark = SparkSession.builder.getOrCreate()
spark

**1. Найти велосипед с максимальным временем пробега.**

In [41]:

trips = spark.read.format('csv').option('header', 'true').load("/content/drive/MyDrive/Colab Notebooks/trips.csv")

In [43]:
# Грруппируем данные по bike_id и ищем максимальное значение продолжительности поездки для каждого велосипеда

bike_max_trip_duration  = (
    trips
    .groupBy('bike_id')
    .agg(
        F.max(F.col("duration").cast(t.IntegerType())).alias("duration")
    )
)

# Сортируем по убыванию длительности
top_longest_trip = bike_max_trip_duration .orderBy(F.col('duration').desc())
# Вывод первого значения с максимальным временем пробега
top_longest_trip.show(1)

+-------+--------+
|bike_id|duration|
+-------+--------+
|    535|17270400|
+-------+--------+
only showing top 1 row



**2. Найти наибольшее геодезическое расстояние между станциями.**

In [49]:
stations = spark.read.format('csv').option('header', 'true').load("/content/drive/MyDrive/Colab Notebooks/stations.csv")

In [50]:
# Выбор столбцов 'id', 'lat' и 'long' и показ первых пяти
stations_data = stations.select("id", "lat", "long")
stations_data.show(5)

# Создание комбинации станций и выбор только различных комбинаций
combo = stations_data.selectExpr('id as A', 'lat as A_lat', 'long as A_long').join(stations_data.selectExpr('id as B', 'lat as B_lat', 'long as B_long'))
dif_combo = combo[combo.A != combo.B]

# Евклидово расстояние между двумя точками
def euclidean_dist(ax, ay, bx, by):
    return sqrt((ax - bx) ** 2 + (ay - by) ** 2)

# Нахождение евклидова расстояния у каждой пары станций и сохранение результатов в RDD
dists = dif_combo.rdd.map(lambda row: (row.A, row.B, euclidean_dist(float(row.A_lat), float(row.A_long), float(row.B_lat), float(row.B_long)) ))

# Нахождение максимального расстояния между станциями
stations_data = dists.max(lambda row: row[2])
stations_data

+---+------------------+-------------------+
| id|               lat|               long|
+---+------------------+-------------------+
|  2|         37.329732|-121.90178200000001|
|  3|         37.330698|        -121.888979|
|  4|         37.333988|        -121.894902|
|  5|         37.331415|          -121.8932|
|  6|37.336721000000004|        -121.894074|
+---+------------------+-------------------+
only showing top 5 rows



('16', '60', 0.7058482821754397)

**3. Найти путь велосипеда с максимальным временем пробега через станции.**

In [51]:
# Выбор столбцов 'id', 'bike_id', 'start_station_id' и 'end_station_id', фильтрация по 'bike_id' равному 535 и сортировка по 'id'
filtered_trips = (
    trips.select("id", "bike_id", "start_station_id", "end_station_id")
    .filter(F.col("bike_id") == 535)
    .orderBy(F.col("id").cast(t.IntegerType()))
)

# Вывод отфильтрованных по условиям данных и количества строк
filtered_trips.show(filtered_trips.count())


+------+-------+----------------+--------------+
|    id|bike_id|start_station_id|end_station_id|
+------+-------+----------------+--------------+
|  4966|    535|              47|            70|
|  5067|    535|              70|            69|
|  5179|    535|              69|            77|
|  5199|    535|              77|            64|
|  7806|    535|              61|            42|
| 11422|    535|              58|            72|
| 12245|    535|              72|            47|
| 12485|    535|              47|            60|
| 12558|    535|              60|            46|
| 13107|    535|              46|            77|
| 13423|    535|              77|            77|
| 14380|    535|              77|            62|
| 14581|    535|              62|            61|
| 15231|    535|              55|            61|
| 15242|    535|              61|            60|
| 15347|    535|              60|            41|
| 15605|    535|              41|            50|
| 15611|    535|    

**4. Найти количество велосипедов в системе**

In [52]:
count_bikes = bike_max_trip_duration.count()

count_bikes

700

**5. Найти пользователей потративших на поездки более 3 часов.**

In [53]:
# Группировка данных по zip_code и вычисление максимальной продолжительности поездки в секундах
# Переименование столбца с максимальной продолжительностью в 'duration'
output_filtered = (
    trips
    .groupBy('zip_code')
    .agg(
        F.max(F.col("duration").cast(t.IntegerType())).alias("duration")
    )
)

# Записи с длительностью поездки более 3 часов
output_filtered = output_filtered.filter(F.col("duration") >= 10800)

# Вывод данных
output_filtered.show()

+--------+--------+
|zip_code|duration|
+--------+--------+
|   94102|  464952|
|   95134|   82487|
|   84606|   14575|
|   80305|   74749|
|   60070|   26540|
|   91910|   20243|
|    2136|   16010|
|   11722|   12173|
|   29454|   14911|
|   94610|   76287|
|   94404|   63504|
|   80301|   36931|
|   94309|   18484|
|   97239|  193241|
|   94592|   26999|
|    7650|   20150|
|   92374|   17156|
|    2464|   27997|
|   11106|   13773|
|   93013|   25116|
+--------+--------+
only showing top 20 rows

